# **TOÁN ỨNG DỤNG VÀ THỐNG KÊ**
## **Project 2 - Đồ án Gram - Schmidt**

## **1. Thông tin sinh viên**
- Họ và tên: Nguyễn Thái Bảo
- Mã số sinh viên: 23120023
- Lớp: 23CTT1
- Email: 23120023@student.hcmus.edu.vn

***
## **2. Giải thuật Gram - Schmidt**

Cho {u1, u2, ..., un} là họ các vector nằm trong không gian Euclid V, từ các vector {u1, u2, ..., un}, ta sẽ xây dựng một họ trực giao {v1, v2, ..., vn} hoặc họ trực chuẩn {q1, q2, ..., qn}.

**a. Cổ điển**

- Bước 1. Đặt v1 = u1, nếu v1 = 0 thì họ không độc lập tuyến tính và kết thúc.

- Bước 2. Đặt v2 = u2 − <u2, v1> / ||v1||^2 * v1  
Nếu v3 = 0 thì họ không độc lập tuyến tính và kết thúc.

- Bước 3. Đặt v3 = u3 − <u3, v1> / ||v1||^2 * v1 − <u3, v2> / ||v2||^2 * v2  
Nếu v2 = 0 thì họ không độc lập tuyến tính và kết thúc.  
Tương tự cho đến vn. Như vậy ta đã xây dựng được một họ các vector trực giao {v1, v2, ..., vn}.

- Bước 4. V Nếu yêu cầu xây dựng họ các vector trực chuẩn, ta tìm các vector trực chuẩn như sau:  
q1 = v1 / ||v1||  
q2 = v2 / ||v2||  
...  
qn = vn / ||vn||

**b. Có điều chỉnh (modified)**

- Bước 1. Đặt q1 - v1 / ||v1||.

- Bước 2. Đặt v2 = u2 − <u2, q1> * q1 và q2 = v2 / ||v2||.

- Bước 3. Đặt v3 = u3 − <u3, q1> * q1 − <u3, q2> * q2 và q3 = v3 / ||v3||.  
Tương tự cho đến vn, qn. Như vậy ta đã xây dựng được một họ các vector trực giao {v1, v2, ..., vn} và họ các vector trực chuẩn {q1, q2, ..., qn}.  
Nếu tồn tại vi = 0 thì họ không độc lập tuyến tính và kết thúc.


***
## **3. Các hàm tiện ích**

In [1]:
# Hàm tính tích vô hướng của hai vector
def dot_product(v1, v2):
    """Tính tích vô hướng của hai vector v1 và v2"""
    return sum(x * y for x, y in zip(v1, v2))

In [2]:
# Hàm tính chuẩn (độ dài) của vector
def norm(v):
    """Tính chuẩn (độ dài) của vector v"""
    return sum(x ** 2 for x in v) ** 0.5

In [3]:
# Hàm nhân hai ma trận
def multiply_matrices(A, B):
    """Nhân hai ma trận A và B"""
    m, n = len(A), len(B[0])  # Số hàng của A và số cột của B
    p = len(B)  # Số hàng của B
    if len(A[0]) != p:
        raise ValueError("Số cột của A không bằng số hàng của B.")
    
    C = [[0] * n for _ in range(m)]  # Khởi tạo ma trận C với giá trị 0
    for i in range(m):
        for j in range(n):
            C[i][j] = sum(A[i][k] * B[k][j] for k in range(p))  # Tính giá trị cho C[i][j]
    
    return C

In [4]:
# Hàm in ma trận
def print_matrix(matrix):
    """In ma trận thẳng hàng, cột"""
    for row in matrix:
        print(row)

In [5]:
# Hàm in ma trận Q, R và kiểm tra A = QR
def print_and_reconstruct(Q, R):
    """In ma trận Q, R và kiểm tra A = QR"""

    print("Ma trận Q:")
    print_matrix(Q)
    print("Ma trận R:")
    print_matrix(R)

    # Kiểm tra A = QR
    A_reconstructed = multiply_matrices(Q, R)
    print("Kiểm tra A = QR:")
    print_matrix(A_reconstructed)

---
## **4. Hàm phân rã QR bằng thuật toán Gram Schmidt**

### **a. Thuật toán Gram Schmidt cổ điển**

In [6]:
def gram_schmidt_classic(A):
    """Thực hiện phân rã QR bằng thuật toán Gram-Schmidt cổ điển"""
    rows, cols = len(A), len(A[0])  # Lấy số hàng và số cột
    Q = [[0] * cols for _ in range(rows)]  # Khởi tạo ma trận Q với giá trị 0
    R = [[0] * cols for _ in range(cols)]  # Khởi tạo ma trận R với giá trị 0
    U = [[A[row][col] for row in range(rows)] for col in range(cols)]  # Lưu các vector cột của A vào U
    V = [[0] * rows for _ in range(cols)]  # Khởi tạo ma trận V với giá trị 0
    Q_cols = []  # Danh sách các vector q1, q2, ..., qn

    for i in range(cols):
        V[i] = U[i][:]  # Sao chép cột i của U vào cột i của V

        # V2 = U2 - <U2, V1> * V1
        # V3 = U3 - <U3, V1> * V1 - <U3, V2> * V2
        # Tổng quát: V[i] = U[i] - sum(<U[i], V[j]> * V[j] cho j từ 0 đến i-1

        for j in range(i):
            coeff = dot_product(U[i], V[j])
            norm_sq = dot_product(V[j], V[j])

            for row in range(rows):
                V[i][row] -= coeff / norm_sq * V[j][row]

        # Nếu V[i] = 0
        if norm(V[i]) == 0:
            raise ValueError("Các cột của A không độc lập tuyến tính.")
        
        norm_V_i = norm(V[i])
        q_i = [V[i][row] / norm_V_i for row in range(rows)]
        
        # Lưu trữ vector q_i vào danh sách Q_cols
        Q_cols.append(q_i)


    # Xây dựng ma trận Q từ danh sách các vector trực chuẩn
    Q = [[Q_cols[j][i] for j in range(cols)] for i in range(rows)]
    
    # Xây dựng ma trận R từ Q và U
    for i in range(cols):
        for j in range(i, cols):
            R[i][j] = dot_product(Q_cols[i], U[j])

    return Q, R

### **b. Thuật toán Gram Schmidt điều chỉnh**

In [7]:
def gram_schmidt_modified(A):
    """Thực hiện phân rã QR bằng thuật toán Gram-Schmidt trực chuẩn hóa"""
    rows, cols = len(A), len(A[0])  # Lấy số hàng và số cột
    Q = [[0] * cols for _ in range(rows)]  # Khởi tạo ma trận Q với giá trị 0
    R = [[0] * cols for _ in range(cols)]  # Khởi tạo ma trận R với giá trị 0
    
    Q_cols = []  # Danh sách các vector q1, q2, ..., qn
    
    for i in range(cols):
        v = [A[row][i] for row in range(rows)]  # Lấy cột thứ i của ma trận A

        for q in Q_cols:
            coeff = dot_product(q, v)
            v = [v[j] - coeff * q[j] for j in range(rows)]  # Trừ thành phần đã chiếu
        
        norm_v = norm(v)
        if norm_v == 0:
            raise ValueError("Các cột của A không độc lập tuyến tính.")
        
        q_i = [x / norm_v for x in v]
        Q_cols.append(q_i)  # Lưu vector trực chuẩn vào danh sách
    
    # Xây dựng ma trận Q từ danh sách các vector trực chuẩn
    Q = [[Q_cols[j][i] for j in range(cols)] for i in range(rows)]
    
    # Xây dựng ma trận R từ Q và A
    for i in range(cols):
        for j in range(i, cols):
            R[i][j] = dot_product([A[row][j] for row in range(rows)], Q_cols[i])
    
    return Q, R

***
## **5. Kiểm tra kết quả**

In [15]:
# Ví dụ 1
A = [[0, 1, 1, 1], [2, -1, 2, 0], [1, 0, 0, 0], [0, 0, -1, 1]]
print("Ma trận A:")
print_matrix(A) 
print()

print("Phân rã QR bằng thuật toán Gram-Schmidt cổ điển:")
Q_A1, R_A1 = gram_schmidt_classic(A)
print_and_reconstruct(Q_A1, R_A1)
print()

print("Phân rã QR bằng thuật toán Gram-Schmidt điều chỉnh:")
Q_A2, R_A2 = gram_schmidt_modified(A)
print_and_reconstruct(Q_A2, R_A2)

Ma trận A:
[0, 1, 1, 1]
[2, -1, 2, 0]
[1, 0, 0, 0]
[0, 0, -1, 1]

Phân rã QR bằng thuật toán Gram-Schmidt cổ điển:
Ma trận Q:
[0.0, 0.9128709291752769, 0.31622776601683783, 0.2581988897471611]
[0.8944271909999159, -0.18257418583505533, 0.31622776601683783, 0.2581988897471611]
[0.4472135954999579, 0.36514837167011077, -0.6324555320336759, -0.5163977794943224]
[0.0, 0.0, -0.6324555320336759, 0.7745966692414834]
Ma trận R:
[2.23606797749979, -0.8944271909999159, 1.7888543819998317, 0.0]
[0, 1.0954451150103321, 0.5477225575051663, 0.9128709291752769]
[0, 0, 1.5811388300841893, -0.31622776601683805]
[0, 0, 0, 1.0327955589886444]
Kiểm tra A = QR:
[0.0, 1.0, 1.0, 1.0]
[2.0, -0.9999999999999999, 1.9999999999999996, -2.7755575615628914e-17]
[1.0, 5.551115123125783e-17, 2.498001805406602e-16, 0.0]
[0.0, 0.0, -0.9999999999999998, 1.0]

Phân rã QR bằng thuật toán Gram-Schmidt điều chỉnh:
Ma trận Q:
[0.0, 0.9128709291752769, 0.31622776601683783, 0.25819888974716104]
[0.8944271909999159, -0.18257418

In [22]:
# Ví dụ 2
A = [[1, -1, 2], [1, 0, -1], [-1, 1, 2], [0, 1, 1]]
print("Ma trận A:")
print_matrix(A) 
print()

print("Phân rã QR bằng thuật toán Gram-Schmidt cổ điển:")
Q_A1, R_A1 = gram_schmidt_classic(A)
print_and_reconstruct(Q_A1, R_A1)
print()

print("Phân rã QR bằng thuật toán Gram-Schmidt điều chỉnh:")
Q_A2, R_A2 = gram_schmidt_modified(A)
print_and_reconstruct(Q_A2, R_A2)

Ma trận A:
[1, -1, 2]
[1, 0, -1]
[-1, 1, 2]
[0, 1, 1]

Phân rã QR bằng thuật toán Gram-Schmidt cổ điển:
Ma trận Q:
[0.5773502691896258, -0.25819888974716115, 0.7745966692414835]
[0.5773502691896258, 0.5163977794943222, -0.25819888974716115]
[-0.5773502691896258, 0.25819888974716115, 0.5163977794943223]
[0.0, 0.7745966692414834, 0.25819888974716115]
Ma trận R:
[1.7320508075688776, -1.1547005383792517, -0.5773502691896258]
[0, 1.2909944487358058, 0.2581988897471612]
[0, 0, 3.098386676965934]
Kiểm tra A = QR:
[1.0000000000000002, -1.0000000000000002, 2.000000000000001]
[1.0000000000000002, -1.1102230246251565e-16, -1.0000000000000004]
[-1.0000000000000002, 1.0000000000000002, 2.0000000000000004]
[0.0, 1.0000000000000002, 1.0000000000000004]

Phân rã QR bằng thuật toán Gram-Schmidt điều chỉnh:
Ma trận Q:
[0.5773502691896258, -0.258198889747161, 0.7745966692414833]
[0.5773502691896258, 0.5163977794943224, -0.25819888974716104]
[-0.5773502691896258, 0.258198889747161, 0.5163977794943222]
[0.

**Nhận xét**:
Khi nhân lại Q và R, có thể có sai số nhỏ do làm tròn, nhân số thực chấm động. Sai số rất nhỏ, bỏ qua sai số thì ta có kết quả A = QR.

***
## **6. Mở rộng: Sử dụng thư viện và so sánh kết quả**

Ta có thể sử dụng hàm `numpy.linalg.qr` trong thư viện NumPy để phân rã QR

In [18]:
# numpy.linalg.qr

import numpy as np
A = np.array([[-1, -1, 1], [1, 3, 3], [-1, -1, 5], [1, 3, 7]])
Q, R = np.linalg.qr(A)

print("Ma trận Q từ numpy:")
print(Q)

print("Ma trận R từ numpy:")
print(R)

print("Nhân Q với R lại để kiểm tra:")
A_reconstructed = np.dot(Q, R)
print(A_reconstructed)

print("A = QR: ")
print(np.allclose(A, A_reconstructed))

Ma trận Q từ numpy:
[[-0.5 -0.5  0.5]
 [ 0.5 -0.5  0.5]
 [-0.5 -0.5 -0.5]
 [ 0.5 -0.5 -0.5]]
Ma trận R từ numpy:
[[ 2.  4.  2.]
 [ 0. -2. -8.]
 [ 0.  0. -4.]]
Nhân Q với R lại để kiểm tra:
[[-1. -1.  1.]
 [ 1.  3.  3.]
 [-1. -1.  5.]
 [ 1.  3.  7.]]
A = QR: 
True


In [11]:
# numpy.linalg.qr

import numpy as np
A = np.array([[1, 1, 2], [2, -1, 1], [-2, 4, 1]])
Q, R = np.linalg.qr(A)

print("Ma trận Q từ numpy:")
print(Q)

print("Ma trận R từ numpy:")
print(R)

print("Nhân Q với R lại để kiểm tra:")
A_reconstructed = np.dot(Q, R)
print(A_reconstructed)

print("A = QR: ")
print(np.allclose(A, A_reconstructed))

Ma trận Q từ numpy:
[[-0.33333333 -0.66666667  0.66666667]
 [-0.66666667 -0.33333333 -0.66666667]
 [ 0.66666667 -0.66666667 -0.33333333]]
Ma trận R từ numpy:
[[-3.          3.         -0.66666667]
 [ 0.         -3.         -2.33333333]
 [ 0.          0.          0.33333333]]
Nhân Q với R lại để kiểm tra:
[[ 1.  1.  2.]
 [ 2. -1.  1.]
 [-2.  4.  1.]]
A = QR: 
True


**Nhận xét**: Về mặt giá trị (tuyệt đối), kết quả của hàm `numpy.linalg.qr` trong thư viện NumPy của Python trả về là **tương đồng** với kết quả của hai thuật toán phân rã QR bằng giải thuật Gram Schmidt cổ điển và có điều chỉnh ở trên. Chỉ có sự khác nhau về dấu của một số cột/dòng, sao cho khi nhân lại kết quả không đổi.

Qua đó, ta rút ra được một nhận xét: kết quả phân rã Q, R từ ma trận A **không là duy nhất**.


***

## **7. Mở rộng: Ứng dụng của QR Decomposition - Phân rã QR**

### **Một số tính chất của QR Decomposition**

QR decomposition là việc phân tách ma trận A kích thước m x n thành: A = QR

Trong đó:
- Q là ma trận m x n có **các vector cột trực chuẩn**, tức là `Q^T.Q = I`, hay `Q^T = Q^-1`.
- R là ma trận n x n **tam giác trên khả nghịch**.


### **Ứng dụng của QR Decomposition**

> QR decomposition được ứng dụng rộng rãi trong *khoa học tính toán, thống kê, học máy và xử lý tín hiệu*. Dưới đây là một số ứng dụng tiêu biểu:

#### **a. Giải hệ phương trình tuyến tính Ax = b**

QR decomposition giúp giải hệ tuyến tính bằng cách:
1. Phân tích  `A = QR `
2. Thay vào phương trình  `Ax = b --> QRx = b `
3. Giải:
   -  `Q^T.b = c `
   -  `Rx = c`  (giải bằng phép thế ngược do  R  là ma trận tam giác trên)

**Ưu điểm**: ổn định số học hơn phương pháp khử Gauss, đặc biệt với ma trận gần suy biến.

#### **b. Phân tích hồi quy tuyến tính (Least Squares)**

Trong bài toán tìm nghiệm tối ưu (tối thiểu hóa sai số bình phương - bình phương nhỏ nhất): `min_x ||Ax - b||^2`


Thay vì giải phương trình chuẩn  `A^T.Ax = A^T.b` , ta dùng phân rã QR:  
`A = QR --> ||Ax - b|| = ||QRx - b|| = ||Rx - Q^T.b||`
 (do Q trực giao)

Giải hệ tam giác trên  `Rx = Q^T.b`

**Ưu điểm**: tránh mất chính xác do nhân chuyển vị  `A^T.A` 

#### **c. Tính trị riêng bằng thuật toán QR (QR Algorithm)**

Sử dụng QR decomposition trong thuật toán `QR iteration` để tìm trị riêng:

1.  `A_k = Q_k.R_k` 
2.  `A_{k+1} = R_k.Q_k`

Lặp lại cho đến khi hội tụ về ma trận tam giác trên với trị riêng nằm trên đường chéo.

**Ứng dụng**: phân tích rung động, xử lý ảnh, nhận dạng mẫu, ...

#### **d. Ứng dụng trong học máy và phân tích dữ liệu**

- Dùng trong phương pháp Phân tích thành phần chính `Principal Component Analysis (PCA)` và Giảm chiều dữ liệu `Dimensionality Reduction`.
- Hỗ trợ trực chuẩn hóa vector cơ sở ==> giảm nhiễu và trích xuất đặc trưng hiệu quả hơn.
- Tăng độ ổn định số học trong các bước tính nội bộ của mô hình học máy trong các thư viện như `scikit-learn`.

#### **e. Ứng dụng trong xử lý tín hiệu số**

- Trong các bộ lọc thích nghi (`adaptive filtering`)
- Hỗ trợ thuật toán RLS (`Recursive Least Squares`)


***
## **8. Mô tả ý tưởng và các hàm**
### **Ý tưởng**

Bài toán yêu cầu phân rã ma trận A thành tích A = QR, trong đó:

- Q là ma trận trực giao (hoặc ma trận gồm các vector cột trực chuẩn nếu A không là ma trận vuông).
- R là ma trận tam giác trên khả nghịch.

Để thực hiện, ta có thể triển khai một trong hai thuật toán:

- `Thuật toán Gram-Schmidt cổ điển`: Dựa trên việc loại bỏ thành phần trực giao của các vector để xây dựng một họ trực giao (v1, v2, ..., vn), sau đó có thể chuẩn hóa để tạo thành họ trực chuẩn (q1, q2, ..., qn).
- `Thuật toán Gram-Schmidt điều chỉnh`: Cải tiến thuật toán cổ điển để giảm sai số số học, đảm bảo độ chính xác cao hơn khi làm việc với các số thực.

Ngoài ra, ta cũng có thể mở rộng so sánh kết quả với hàm `numpy.linalg.qr` từ thư viện NumPy của Python để kiểm tra tính chính xác.

### **Thuật giải phân rã QR - QR decomposition**
- Bước 1. Xác định n cột của A = [u1 | u2 | ... | un].

- Bước 2. Thực hiện thuật giải Gram - Schmidt trực chuẩn hóa u1, u2,..., un. Thông báo nếu các cột của A không độc lập tuyến tính và kết thúc,ngược lại sẽ nhận được q1, q2, ..., qn là họ trực chuẩn tương ứng.

- Bước 3. Xây dựng ma trận Q gồm n cột q1, q2, ..., qn.

- Bước 4. Xây dựng ma trận R kích thước n × n có dạng:  
[<u1, q1>   <u2, q1>    ...     <un, q1>]  
[0          <u2, q2>    ...     <un, q2>]  
.           .           .       .  
.           .            .      .  
.           .             .     .  
[0          0           ...     <un, qn>]  

### **Mô tả các hàm**

**1. Hàm `dot_product(v1, v2)`**
> Tính tính vô hướng của hai vector v1 và v2

**2. Hàm `norm(v)`**
> Tính chuẩn (độ dài) của vector v

**3. Hàm `multiply_matrices(A, B)`**
> Thực hiện nhân hai ma trận

**4. Hàm `print_and_reconstruct(A, Q, R)`**
> In ra ma trận Q, R và nhân lại để kiểm tra

**5. Hàm `print_matrix(matrix)`**
> In ma trận

**6. Hàm `gram_schmidt_classic(A)`**
> Phân rã QR bằng thuật toán Gram - Schmidt cổ điển, trả về Q và R

**7. Hàm `gram_schmidt_modified(A)`**
> Phân rã QR bằng thuật toán Gram - Schmidt có điều chỉnh, trả về Q và R

**8. Hàm `numpy.linalg.qr(A)`**
> Phân rã QR sử dụng hàm của thư viện NumPy, ngôn ngữ Python, trả về Q và R để kiểm tra kết quả (Mở rộng)